In [1]:
import tensorflow as tf
import matplotlib
import keras.backend as K
import os
import time

## Training 

Note that this model was trained on google colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
IMAGE_SHAPE = (496, 369)
TRAINING_DATA_DIR = r'/content/drive/MyDrive/DSP/A2/training'
VALID_DATA_DIR = r'/content/drive/MyDrive/DSP/A2/validation'

In [4]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
train_generator = datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    shuffle=True,
    target_size=IMAGE_SHAPE,
)
valid_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=False,
    target_size=IMAGE_SHAPE,
)

Found 312 images belonging to 2 classes.
Found 78 images belonging to 2 classes.


In [5]:
def build_model(num_classes):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', 
                           input_shape=(IMAGE_SHAPE[0], IMAGE_SHAPE[1], 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model
model = build_model(num_classes=2)

In [6]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[get_f1]
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 494, 367, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 247, 183, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 245, 181, 16)      2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 122, 90, 16)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 120, 88, 32)       4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 60, 44, 32)       0

In [18]:
EPOCHS = 200
BATCH_SIZE = 32
# Here I will use model checkpoint callback so that I can save the models with the highest validation f1 score and load it
checkpoint_filepath = '/content/drive/MyDrive/DSP/A2/checkpoint/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_get_f1',
    mode='max',
    save_best_only=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    validation_steps= valid_generator.samples // BATCH_SIZE,
                    verbose=1, callbacks=[model_checkpoint_callback]
                    )

Epoch 1/100
9/9 [==============================] - 4s 470ms/step - loss: 0.1450 - get_f1: 0.9294 - val_loss: 2.9606 - val_get_f1: 0.8750
Epoch 2/100
9/9 [==============================] - 4s 407ms/step - loss: 0.1398 - get_f1: 0.9375 - val_loss: 2.8273 - val_get_f1: 0.8594
Epoch 3/100
9/9 [==============================] - 5s 532ms/step - loss: 0.1102 - get_f1: 0.9549 - val_loss: 2.6137 - val_get_f1: 0.8281
Epoch 4/100
9/9 [==============================] - 4s 408ms/step - loss: 0.1202 - get_f1: 0.9456 - val_loss: 2.4231 - val_get_f1: 0.8750
Epoch 5/100
9/9 [==============================] - 4s 446ms/step - loss: 0.1159 - get_f1: 0.9583 - val_loss: 2.4187 - val_get_f1: 0.8750
Epoch 6/100
9/9 [==============================] - 4s 502ms/step - loss: 0.1109 - get_f1: 0.9653 - val_loss: 1.8249 - val_get_f1: 0.8594
Epoch 7/100
9/9 [==============================] - 4s 464ms/step - loss: 0.0958 - get_f1: 0.9722 - val_loss: 2.1437 - val_get_f1: 0.8594
Epoch 8/100
9/9 [========================

In [19]:
model.load_weights(checkpoint_filepath)

In [20]:
# Manually testing the model to see which validation was picked up
import os
from sklearn.metrics import f1_score
import numpy as np
y_true = []
y_pred = []
for file in os.listdir(r'/content/drive/MyDrive/DSP/A2/validation/n0'):
  image = tf.keras.preprocessing.image.load_img(rf'/content/drive/MyDrive/DSP/A2/validation/n0/{file}',
                                            target_size=IMAGE_SHAPE)
  input_arr = tf.keras.preprocessing.image.img_to_array(image)
  input_arr = np.array([input_arr])  # Convert single image to a batch.
  input_arr = input_arr.astype('float32') / 255.

  predictions = model.predict(input_arr)

  if predictions[0][0] > predictions[0][1]:
    y_pred.append(0)
  else:
    y_pred.append(1)

  y_true.append(0)

for file in os.listdir(r'/content/drive/MyDrive/DSP/A2/validation/n1'):
  image = tf.keras.preprocessing.image.load_img(rf'/content/drive/MyDrive/DSP/A2/validation/n1/{file}',
                                            target_size=IMAGE_SHAPE)
  input_arr = tf.keras.preprocessing.image.img_to_array(image)
  input_arr = np.array([input_arr])  # Convert single image to a batch.
  input_arr = input_arr.astype('float32') / 255.

  predictions = model.predict(input_arr)

  if predictions[0][0] > predictions[0][1]:
    y_pred.append(0)
  else:
    y_pred.append(1)

  y_true.append(1)

f1_score(y_true=y_true, y_pred=y_pred)

1/1 [==============================] - 0s 31ms/step


0.898550724637681

In [23]:
model.save('/content/drive/MyDrive/DSP/A2/model')